In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# strainforfard peaks search
import plotly.graph_objects as go
from scipy.signal import find_peaks, savgol_filter

# peaks search by gauss decomposition
import gausspy
import gausspy.gp as gp
import pickle

In [ ]:
# ga_file = "/content/drive/MyDrive/AIRI/RamanSpectra/ga_2_3_5_6.csv"
# fg_file = "/content/drive/MyDrive/AIRI/RamanSpectra/fg_1-5_7-11.csv"
ga_file = "dataSrc/ga_2_3_5_6.csv"
fg_file = "dataSrc/fg_1-5_7-11.csv"
data_ga = pd.read_csv(ga_file, sep=";")
print(f"data with GA samples shape: {data_ga.shape}")
data_fg = pd.read_csv(fg_file, sep=";")
print(f"data with FG samples shape: {data_fg.shape}")

In [ ]:
y_fg = data_fg['class'].values
X_fg = data_fg.drop(columns=['class'])
print(f"shapes of X and y of FG samples respectively is: {X_fg.shape}, {y_fg.shape}")

y_ga = data_ga['class'].values
X_ga = data_ga.drop(columns=['class'])
print(f"shapes of X and y of GA samples respectively is: {X_ga.shape}, {y_ga.shape}")

In [ ]:
print(f"old fg classes: {set(y_fg)}")
for index, element in enumerate(set(y_fg)):
    y_fg[y_fg == element] = index
print(f"new fg classes: {set(y_fg)}\n")

print(f"old ga classes: {set(y_ga)}")
for index, element in enumerate(set(y_ga)):
    y_ga[y_ga == element] = index + len(set(y_fg))
print(f"new ga classes: {set(y_ga)}")

In [ ]:
group1 = [0, 1, 2, 3, 4]
group2 = [5, 6, 7, 8, 9]
group3 = [10, 11]
group4 = [12, 13]
groups = [group1, group2, group3, group4]

def add_group(y, groups, add=0):
    new_y = np.zeros((y.shape[0], 2))
    new_y[:, 1] = y
    for group_num, group in enumerate(groups):
        for class_num in group:
            new_y[y==class_num] = np.array([group_num + add, class_num])
    return new_y

y_ga = add_group(y_ga, (group3, group4), add=2)
y_fg = add_group(y_fg, (group1, group2))
print(f"Now, shapes of y_fg and ga is respectively: {y_ga.shape}, {y_fg.shape}")

In [ ]:
y = np.concatenate((y_fg, y_ga), axis=0)
X = np.concatenate((X_fg, X_ga), axis=0)
X = X.reshape(X.shape[0], 1, -1)
print(f"shapes of X and y is respectively: {X.shape}, {y.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
data_loader_train = DataLoader(tuple(zip(torch.tensor(X_train).float(), torch.tensor(y_train).long())), 
                                      batch_size=16, 
                                      shuffle=True)
data_loader_test = DataLoader(tuple(zip(torch.tensor(X_test).float(), torch.tensor(y_test).long())), 
                                     batch_size=16, 
                                     shuffle=False)

## Неудачное разложение на гауссианы:

In [ ]:
with open(FILENAME, 'w'):
    pass

with open(FILENAME_DECOMP, 'w'):
    pass

# Data properties
RMS = 1
NCHANNELS = 994
FILENAME = 'simple_gaussian.pickle'

# Component properties

# Initialize
data = {}
# chan = np.arange(NCHANNELS)
chan = data_fg.columns[1:].values.astype(float)
# [float(digit) for digit in data_fg.columns[1:].values]
errors = np.ones(NCHANNELS) * RMS

# spectrum = np.random.randn(NCHANNELS) * RMS
# spectrum += gaussian(AMP, FWHM, MEAN)(chan)
spectrum = savgol_filter(X[-1].reshape(-1), 15, 2)
# savgol_filter(X[-1].reshape(-1), 75, 2)
print(spectrum.shape, chan.shape)

# Enter results into AGD dataset
data['data_list'] = [spectrum]
data['x_values'] = [chan]
data['errors'] = [errors]


pickle.dump(data, open(FILENAME, 'wb'))

In [ ]:
# smoothing parameter
alpha1 = 0.05
# the signal-to-noise ratio threshold below which amplitude GaussPy will not fit a component.
snr_thresh = 1
FILENAME = 'simple_gaussian.pickle'
FILENAME_DECOMP = 'simple_gaussian_decomposed.pickle'

# Load GaussPy
g = gp.GaussianDecomposer()

# Setting AGD parameters
g.set('phase', 'one')
g.set('SNR_thresh', [snr_thresh, snr_thresh])
g.set('alpha1', alpha1)

# Run GaussPy
data_decomp = g.batch_decomposition(FILENAME)

# Save decomposition information
pickle.dump(data_decomp, open(FILENAME_DECOMP, 'wb'))

In [ ]:
def gaussian(amp, fwhm, mean):
    return lambda x: amp * np.exp(-4. * np.log(2) * (x-mean)**2 / fwhm**2)

def unravel(List):
    return np.array([i for array in List for i in array])

FILENAME = 'simple_gaussian.pickle'
FILENAME_DECOMP = 'simple_gaussian_decomposed.pickle'

data = pickle.load(open(FILENAME, 'rb'))
spectrum = unravel(data['data_list'])
chan = unravel(data['x_values'])
errors = unravel(data['errors'])

data_decomp = pickle.load(open(FILENAME_DECOMP, 'rb'))
means_fit = unravel(data_decomp['means_fit'])
amps_fit = unravel(data_decomp['amplitudes_fit'])
fwhms_fit = unravel(data_decomp['fwhms_fit'])

fig = plt.figure(figsize=(10, 6), dpi=300)
ax = fig.add_subplot(111)

model = np.zeros(len(chan))

for j in range(len(means_fit)):
    component = gaussian(amps_fit[j], fwhms_fit[j], means_fit[j])(chan)
    model += component
    ax.plot(chan, component, color='red', lw=1.5)

ax.plot(chan, spectrum, label='Data', color='black', linewidth=1.5)
ax.plot(chan, model, label = f'$log \\alpha={alpha1}$', color='purple', linewidth=2.)
# ax.plot(chan, errors, label = 'Errors', color='green', linestyle='dashed', linewidth=2.)

ax.set_xlabel('Channels', fontsize=15)
ax.set_ylabel('Amplitude', fontsize=15)

ax.set_xlim(chan.min()-30,chan.max()+30)
ax.set_ylim(np.min(spectrum),np.max(spectrum))
ax.legend(loc=2)

plt.show()

In [ ]:
max_len = 0
max_array = []
max_alpha = 0
max_snr = 0
for j in np.linspace(0.2, 3, 15):
    for i in np.linspace(0.1, 1, 10):
        alpha1 = i
        # the signal-to-noise ratio threshold below which amplitude GaussPy will not fit a component.
        snr_thresh = j
        FILENAME = 'simple_gaussian.pickle'
        FILENAME_DECOMP = 'simple_gaussian_decomposed.pickle'

        # Load GaussPy
        g = gp.GaussianDecomposer()

        # Setting AGD parameters
        g.set('phase', 'one')
        g.set('SNR_thresh', [snr_thresh, snr_thresh])
        g.set('alpha1', alpha1)

        # Run GaussPy
        data_decomp = g.batch_decomposition(FILENAME)

        # Save decomposition information
        pickle.dump(data_decomp, open(FILENAME_DECOMP, 'wb'))

        data = pickle.load(open(FILENAME, 'rb'))
        spectrum = unravel(data['data_list'])
        chan = unravel(data['x_values'])
        errors = unravel(data['errors'])

        data_decomp = pickle.load(open(FILENAME_DECOMP, 'rb'))
        means_fit = unravel(data_decomp['means_fit'])
        amps_fit = unravel(data_decomp['amplitudes_fit'])
        fwhms_fit = unravel(data_decomp['fwhms_fit'])

    #     fig = plt.figure(figsize=(10, 6), dpi=300)
    #     ax = fig.add_subplot(111)

    #     model = np.zeros(len(chan))

    #     for j in range(len(means_fit)):
    #         component = gaussian(amps_fit[j], fwhms_fit[j], means_fit[j])(chan)
    #         model += component
    #         ax.plot(chan, component, color='red', lw=1.5)

    #     ax.plot(chan, spectrum, label='Data', color='black', linewidth=1.5)
    #     ax.plot(chan, model, label = f'$log \\alpha={alpha1}$', color='purple', linewidth=2.)
        # ax.plot(chan, errors, label = 'Errors', color='green', linestyle='dashed', linewidth=2.)

    #     ax.set_xlabel('Channels', fontsize=15)
    #     ax.set_ylabel('Amplitude', fontsize=15)
    #     ax.set_xlim(chan.min()-30,chan.max()+30)
    #     ax.set_ylim(np.min(spectrum),np.max(spectrum))
    #     ax.legend(loc=2)
        if len(means_fit) > max_len:
            max_len = len(means_fit)
            max_array = means_fit
            max_alpha = alpha1
            max_snr = snr_thresh
        print(f"alpha = {alpha1}, snr_thresh = {snr_thresh}, means_fit:\n{means_fit}")
    #     plt.show()
print(max_len, max_alpha, max_snr, max_array)

In [ ]:
print(max_len, max_array)